<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
OptiBot Chatbot Controller

This module provides the main backend logic for the OptiLine virtual assistant chatbot ("OptiBot").
The chatbot can answer user questions about the OptiLine system, leveraging both generative AI (Gemini) and live data from Firebase when needed.

Functions:
    - ask_optibot(question: str) -> str
        Main entry point. Receives a user question (string) and returns a single English answer as a string.
        Answers can include both real-time data from the system and general knowledge.

Dependencies:
    - google-generativeai (for Gemini API)
    - importnb (for loading other project notebooks as modules)
    - Firebase helper modules (existing: FireBase, user_controller, etc.)
    - Colab secrets (for securely storing Gemini API key)

Note:
    This module is backend-only; it does not include any frontend or UI logic.
    All code comments and docstrings are in English for documentation purposes.
"""

In [ ]:
# ─── Imports ───

import google.generativeai as genai
from google.colab import userdata
import time
import re
import difflib
# Import project modules using importnb
from importnb import Notebook

# Load Firebase and helper modules as notebooks
with Notebook():
    import FireBase as fb           # Firebase data access functions
    import user_controller as uc    # User management and leaderboard functions
    import mqqt_sim_indoor as indoor    # Indoor sensor simulator
    import mqqt_sim_outdoor as outdoor  # Outdoor sensor simulator

# ─── Constants ───

# --- OptiBot Official Documentation Context (Knowledge Base) ---
OPTI_BOT_CONTEXT = """
The CIM & Robotics Laboratory at Braude College of Engineering, established in 1997, serves as an advanced learning environment for students in Industrial and Mechanical Engineering. Located in Room D106, the lab includes semi-industrial CIM (Computer Integrated Manufacturing) and FMS (Flexible Manufacturing System) platforms.

The facility offers hands-on experience with:
- Robotic arms and assembly systems
- CNC machining and simulation
- CAD/CAM tools
- Vision-based quality control
- Programmable logic controllers (PLCs)
- Automated storage and retrieval systems (AS/RS)

Students engage in real-world manufacturing scenarios that bridge theoretical knowledge with industry-relevant skills.

The OptiLine Dashboard is a cloud-based interface designed for engineers working with the autonomous production line in the CIM lab. It provides:
- Real-time sensor monitoring from indoor and outdoor environments
- Historical data analysis with trend visualization
- Search capabilities over MQTT-related technical content
- A fault simulator with daily challenges for diagnosing simulated malfunctions
- A gamified experience with scoring based on performance and efficiency
- A user directory and personalized dashboard view

The system is accessible from any device with a secure login and supports both live and simulated data modes.

Available Sensors:
Temperature (Indoor & Outdoor, °C)
Humidity (Indoor & Outdoor, %)
Pressure (Indoor, hPa)
Dlight (Outdoor, Lux)
Distance (Indoor, mm, detects movement via ultrasound)
All sensors stream data in real time via MQTT protocol.

Points are earned by completing challenges in the Fault Simulator.
More difficult faults = more points.
Faster completion = better leaderboard standing.
You must complete all steps to get points; no partial credit.
Your total score accumulates and determines your leaderboard rank.

FAQ
Q: Can I skip steps in the Fault Simulator and still get points?
A: No. You must complete all required steps before submitting. Partial credit is not available.

Q: Can I view or repeat previous faults?
A: Not at this stage. Handled faults cannot be revisited.

Q: Are the sensors live or simulated?
A: Both modes are supported. In simulation mode, realistic data is streamed from a preloaded scenario.

Q: How often is the leaderboard updated?
A: The leaderboard updates after each completed challenge.

The dashboard consists of several main sections:
- Live Sensors Dashboard: Monitor real-time data from indoor and outdoor sensors.
- Statistics Panel: Analyze historical data using interactive plots. Select a date and time window to compare trends and spot irregular patterns.
- MQTT Search Engine: Search technical content directly from the MQTT.org website.
- Fault Simulator: Practice diagnosing simulated production malfunctions and improve your troubleshooting skills.
- User Directory: View the contact information of authorized users in the system.
- Leaderboard: Track your cumulative performance points compared to other engineers in the lab.
"""



# The Gemini API key is securely retrieved from Colab secrets
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")


In [ ]:
def ask_optibot(message: str, chat_history: list) -> tuple:
    """
    OptiBot FAQ: Answers only based on official OptiLine documentation and user guide.
    Does not generate answers outside of the provided knowledge base.

    Args:
        message (str): The latest user message.
        chat_history (list): Previous chat messages, each as a dict (role/content).

    Returns:
        tuple: ("", updated_chat_history)
    """

    # השתמש בקונטקסט הגלובלי שנמצא מחוץ לפונקציה
    OPTILINE_KB = OPTI_BOT_CONTEXT

    WELCOME_MSG = (
        "👋 Hello! I'm OptiBot, your official FAQ assistant for the OptiLine Dashboard.\n\n"
        "I can answer questions strictly based on the CIM & Robotics Lab and OptiLine documentation below.\n"
        "Ask about:\n"
        "- The CIM & Robotics Lab\n"
        "- What sensors are available\n"
        "- How the Fault Simulator and leaderboard work\n"
        "- How to use the dashboard sections\n"
        "- Anything from the official user guide!\n\n"
        "I will always answer only according to the official information – no improvising.\n"
        "Type your question below."
    )

    # Start with welcome if new chat
    if not chat_history or len(chat_history) == 0:
        chat_history = [{"role": "assistant", "content": WELCOME_MSG}]
    chat_history.append({"role": "user", "content": message})

    msg = message.lower().strip()

    # --- Fuzzy keywords for documentation topics ---
    FAQ_MAP = {
        "lab": [
            "what is cim", "cim lab", "robotics lab", "about the lab"
        ],
        "sensors": [
            "available sensors", "what sensors", "list sensors", "sensor types"
        ],
        "points": [
            "how to earn points", "how do i earn points", "score system", "leaderboard", "how do i get points"
        ],
        "skip steps": [
            "can i skip steps", "partial credit"
        ],
        "repeat faults": [
            "repeat previous faults", "view previous faults"
        ],
        "live or simulated": [
            "live or simulated", "are the sensors live", "simulated sensors"
        ],
        "leaderboard update": [
            "how often leaderboard", "leaderboard update"
        ],
        "usage": [
            "how to use", "user guide", "help", "how do i"
        ],
        "about app": [
            "about the application", "what is optiline", "about optiline"
        ],
        "optimize": [
            "optimize", "how do i optimize"
        ],
    }

    # --- Answers mapped to keywords ---
    FAQ_ANSWERS = {
        "lab": (
            "The CIM & Robotics Laboratory at Braude College of Engineering was established in 1997. "
            "It serves as an advanced learning environment for students in Industrial and Mechanical Engineering. "
            "Located in Room D106, the lab features semi-industrial CIM and FMS platforms, robotic arms, CNC simulation, "
            "CAD/CAM, vision-based QC, PLCs, and more."
        ),
        "sensors": (
            "The following sensors are integrated into the system:\n"
            "- Temperature (Indoor & Outdoor, °C)\n"
            "- Humidity (Indoor & Outdoor, %)\n"
            "- Pressure (Indoor, hPa)\n"
            "- Dlight (Outdoor, Lux)\n"
            "- Distance (Indoor, mm, detects movement via ultrasound)\n"
            "All sensors stream data in real time via MQTT protocol."
        ),
        "points": (
            "Points are earned by completing Fault Simulator challenges. More difficult and realistic faults yield higher scores. "
            "Faster completion gives you a better standing. You must complete all steps to get points; no partial credit. "
            "Your cumulative score determines your position on the leaderboard, which updates after each completed challenge."
        ),
        "skip steps": (
            "No. You must complete all required steps before submitting. Partial credit is not available in the Fault Simulator."
        ),
        "repeat faults": (
            "Not at this stage. Handled faults cannot be revisited."
        ),
        "live or simulated": (
            "Both modes are supported. In simulation mode, realistic data is streamed from a preloaded scenario. "
            "Otherwise, live sensor data is shown."
        ),
        "leaderboard update": (
            "The leaderboard updates after each completed challenge."
        ),
        "usage": (
            "The dashboard consists of several sections: Live Sensors Dashboard, Statistics Panel, MQTT Search Engine, "
            "Fault Simulator, User Directory, and Leaderboard. Use each section to monitor, analyze, or practice on the system. "
            "Refer to the official user guide in the system for details."
        ),
        "about app": (
            "The OptiLine Dashboard is a cloud-based interface for engineers in the CIM lab. It provides real-time sensor monitoring, "
            "historical data, MQTT search, a fault simulator with challenges, a gamified experience, a user directory, and secure login. "
            "It supports both live and simulated data."
        ),
        "optimize": (
            "To optimize your standing, complete Fault Simulator challenges efficiently and accurately. "
            "Time efficiency and complexity both affect your score and leaderboard position."
        )
    }

    # Flatten keywords and map to keys
    all_keys = []
    key_map = {}
    for topic, keys in FAQ_MAP.items():
        all_keys.extend(keys)
        for k in keys:
            key_map[k] = topic

    # Try direct match first
    for k, topic in key_map.items():
        if k in msg:
            answer = FAQ_ANSWERS[topic]
            chat_history.append({"role": "assistant", "content": answer})
            return "", chat_history

    # Fuzzy match for typos and inexact questions
    closest = difflib.get_close_matches(msg, all_keys, n=1, cutoff=0.7)
    if closest:
        topic = key_map[closest[0]]
        answer = FAQ_ANSWERS[topic]
        chat_history.append({"role": "assistant", "content": answer})
        return "", chat_history

    # Fallback: search for any FAQ-related keyword in the message
    if any(w in msg for w in ["fault", "challenge", "points", "score", "leaderboard", "sensor", "lab", "dashboard", "statistics", "user directory", "mqtt"]):
        # Very simple: search for relevant lines in KB
        matches = [line for line in OPTILINE_KB.split("\n") if any(w in line.lower() for w in msg.split())]
        if matches:
            answer = " ".join(matches)
        else:
            answer = (
                "Please refer to the official documentation above. "
                "If you have a specific question about a feature or policy, please rephrase."
            )
        chat_history.append({"role": "assistant", "content": answer})
        return "", chat_history

    # Not found – generic refusal
    answer = (
        "I can answer only according to the official OptiLine documentation and user guide. "
        "Please ask about system features, usage, scoring, or sensors as described in the official information."
    )
    chat_history.append({"role": "assistant", "content": answer})
    return "", chat_history